# Create Azure Workspace for AI Model Manager

When you want to automate your machine learning workflow with steps like training, packaging a model or deploying an Edge Configuration Pipeline, 
it comes handy to create them in an Azure Machine Learning Studio environment.  
The main goal of this notebook is to create a connection between Azure automation and make it possible to deploy a model into Edge environment via AI Model Manager.  
What makes it possible is a messaging connection between Azure workspace and AI Model Manager. This connection requires an Azure IoT Hub created with a device which represents AI Model Manager in your Edge environment.  
As you will work with sensitive information, you should use this notebook on your local machine and store your keys in a safe way.

By the end of this notebook study, you will be able to
- create required IoT Hub with a Device in Azure
- configure an AI Model Manager workspace for Azure

But first, let's import some required Python packages!

In [ ]:
import os
from pathlib import Path
import stat
import urllib.request

## Create Azure IoT HUB and Device

To understand the messaging concept, please study the relevant Azure pages on [IoT Hub concept](https://learn.microsoft.com/en-us/azure/iot-hub/iot-hub-devguide?source=recommendations).  

### Create an IoT Hub
**&#9757; Manual step!**  
As the _manual steps_ on page [Create an IoT hub](https://learn.microsoft.com/en-us/azure/iot-hub/iot-hub-create-through-portal) are explained in depth, we only focus on how to use the certificates and keys for successful communication.  
Please, create the IoT Hub using the [creation form](https://portal.azure.com/#create/Microsoft.IotHub) after logging in with your Azure user.  
Here you need to choose 
- one of your 'Subscription's, 
- an existing 'Resource Group', or creating a new one, 
- the 'Region' where your resources will be registered, 
- and an 'IoT hub name' of your choice.  

You can leave the other settings on default.

Once the IoT Hub is created, you will have its name, which will be needed later in this notebook, so set the `iot_hub_name` variable.  
Based on the name a hostname and a username will also be connected to your IoT Hub which will be used in this example, as well.  
Please find your hostname on `Properties` page of your Iot Hub as `Host name` and set the `iot_host_name` variable below.


In [ ]:
iot_hub_name = "<iot_hub_name>"
iot_host_name = "<iot_host_name>"

### CA Certificates for IoT Hub

Once the IoT Hub is created, it has a certificate chain that is used to encrypt the communication channel. Model Manager needs to verificate the server's certificate chain, thus it needs its root certificate authority. The certificate authority can differ for the different regions.  
In our example `DigiCert Global Root G2` will be used, but in case of Azure China `DigiCert Global Root CA` should be used. 

For further details of certificate authorities see [DigiCert](https://www.digicert.com/kb/digicert-root-certificates.htm) page. The code cell below uses this page to download the `DigiCert Global Root G2` certificate file.  
To store certificates and keys in one place, you need to define
- `certificates_path`, which should be a path of your choice.  
    We suggest to store your certificates and private keys in a hidden folder of your _'Home'_ directory on your local computer.  
    To do so it is also recommended to execute this notebook locally.

In [ ]:
certificates_path = Path.home() / ".certificates"  # to store your security keys separately, you should define a safe path in your HOME directory

if not certificates_path.is_dir():
    certificates_path.mkdir(parents=True, exist_ok=True)
    os.chmod(certificates_path, stat.S_IRWXU)  # to make the folder safe, change its access rights

os.chdir(certificates_path)

In [ ]:
root_ca_cert_path = certificates_path / "DigiCertGlobalRootG2.crt.pem"
urllib.request.urlretrieve("https://cacerts.digicert.com/DigiCertGlobalRootG2.crt.pem", root_ca_cert_path)

### Create Device Credentials

In our example, for the sake of simplicity we are using Self Signed Certificates in the communication of AI Model Manager and the IoT Hub.

AI Model Manager will use device credentials to identify itself in Azure, and we will create them below.  
The certificates and keys will be created in the previously defined folder `certificates_path`.


For key creation you need to define
- `device_id`,  
    which the name of the device you need to add to your IoT Hub later
- `crt_subj`,  
    which is the subject of the certificate and the device id as _Common Name (CN)_.  
    The other fields of the subject are based on your location (e.g. '/C=IE/S=Dublin/L=Dublin 4/O=Company Ltd./OU=R&D/CN=ic_target_device').  
    A detailed description about used credentials can be found on the [Certificate requirements](https://learn.microsoft.com/en-us/azure/databox-online/azure-stack-edge-gpu-certificate-requirements) Azure pages.


In [ ]:
device_id = "ic_target_device"
device_crt_subject = f"/CN={device_id}"


The following path variables are compiled from the `device_id` and contain the path of the `private key`, the `self-signed device certificate`, and the `fingerprint of the certificate`, respectively.

In [ ]:
key_file_path = certificates_path / f"{device_id}.key"
cert_file_path = certificates_path / f"{device_id}.cert.pem"
thumb_file_path = certificates_path / f"{device_id}.thumb.txt"

Now you can create the certificate and private key files with one `openssl` command.  
In order the command to work, `openssl` must be installed and added to the system PATH.

In [ ]:
os.system(f"openssl req -x509 -sha256 -nodes -newkey rsa:4096 -keyout {key_file_path} -days 730 -out {cert_file_path} -subj '{device_crt_subject}'")

Then let's calculate the fingerprint of the certificate!

In [ ]:
os.system(f"openssl x509 -in {cert_file_path} -noout -fingerprint -sha256 -inform pem > {thumb_file_path}")
fingerprint = open(thumb_file_path).readline()
print("device_id:", device_id)
print("thumbprint:", fingerprint.split("=")[1].replace(":",""))

**&#9757; Manual step!**  
Now you can add your device _manually_ to your IoT Hub by choosing `Devices` menu item and clicking on the `Add` button on the top.  
On the form you need to add 
- `Device ID`,
    which is same as defined above as `device_id`
- `Authentication type` as `X.509 Self-Signed`  
    because we created a self-signed certificate above
- `Primary and Secondary Thumbprint`  
    which is calculated above

### Create MQTT config file for AI Model Manager

To enable the connection between the created IoT Device and AI Model Manager, you need a configuration file which contains security properties for the device.  
This configuration file is compiled the code box below.

In [ ]:
device_crt = open(cert_file_path).read()
device_key = open(key_file_path).read()
root_ca_content = "\n".join(open(root_ca_cert_path).readlines())

endpoint = f'wss://{iot_host_name}:443/$iothub/websocket'  # using WebSocket
# endpoint = f'tls://{iot_host_name}:8883'  # using TCP over TLS

mqtt_config = {
    'ca_cert': root_ca_content,
    'x509_certificate': device_crt,
    'x509_private_key': device_key,
    'endpoint': endpoint,
    'topic_prefix': f'devices/{device_id}',
    'username': f'{iot_host_name}/{device_id}/?api-version=2021-04-12',
    'client_id': device_id
}
mqtt_config

In this step we save the `credential` dictionary as json in the `certificates_path` directory as `mqtt-config.json`.  
When you are about to create a workspace for Azure in AI Model Manager, you have to upload this file.

In [ ]:
import json
with open(certificates_path / f"{device_id}-config.json", "w") as conf:
    json.dump(mqtt_config, conf, indent=4)

## Define Azure workspace in Model Manager
<!-- _source: https://157.163.16.25/mlmanager/assets/docs/user-manual/en-US/index.html#treeId=7d08e4300a8dce92704053b12cb03245_ -->

**&#129306; Please wait!**  
Before you go further in this notebook, you should wait until your device_id appears in the list on the IoT Hub management page under the `Devices` menu.

**&#9757; Manual step!**  
Here we create an Azure workspace in your AI Model Manager running on your Edge Device, and connect it to your previously created device by the compiled `mqtt-config.json` file.

The steps have to be taken:
1. Open `Workspace area` and click on the `Create new workspace` button
1. ​Enter the name of your new workspace
1. ​Choose the workspace type `Cloud Integration`
1. ​Choose the Cloud system `Microsoft Azure`
1. ​Choose the Integration type `IoT Hub`
1. ​Upload the MQTT connection configuration file (created above)
1. ​Click on the `Create workspace` button

![aimm-create-new](images/aimm-workspace-new.png)



Once the workspace is created in AI Model Manager, a couple of messages should appear on the `Overview` page of your IoT Hub.

## Useful links

- Install Azure CLI  
[https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-linux](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-linux)
- Create IoT HUB and Device  
[https://learn.microsoft.com/en-us/azure/iot-develop/quickstart-send-telemetry-iot-hub](https://learn.microsoft.com/en-us/azure/iot-develop/quickstart-send-telemetry-iot-hub)
